In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.insert(0, '../../')
from src.encode_data import *
from src.midi_data import *
# from src.data_sources import process_parallel, transform_csv_row
from src.midi_transform import *
from concurrent.futures import ProcessPoolExecutor
from fastprogress.fastprogress import master_bar, progress_bar

In [3]:
import traceback
import time

In [4]:
from tqdm import tqdm

## Standardize and reformat raw midi files before encoding to text
- Transform key to C major
- Remove unused instruments
- Combine multiple tracks with the same instrument into a single part
- Melody, Piano, String

### Load midi data

In [5]:
version = 'v13'
data_path = Path('data/midi')
version_path = data_path/version

In [6]:
import pandas as pd

In [7]:
source_dir = 'midi_sources'
out_dir = 'piano_duet'
source_csv = version_path/'metadata'/f'{source_dir}.csv'
out_csv = version_path/out_dir/f'{out_dir}.csv'
out_csv.parent.mkdir(parents=True, exist_ok=True)
source_csv, out_csv

(PosixPath('data/midi/v13/metadata/midi_sources.csv'),
 PosixPath('data/midi/v13/piano_duet/piano_duet.csv'))

In [8]:
num_comps = 2 # note, duration
cutoff = 4 # max instruments
max_dur = 128

### Pebble

In [9]:
from pebble import ProcessPool
from concurrent.futures import TimeoutError

In [10]:
def process_all(func, arr, total=None, max_workers=None, timeout=None):
    with ProcessPool() as pool:
        future = pool.map(func, arr, timeout=timeout)

        iterator = future.result()
        results = []
        for i in progress_bar(range(len(arr)), total=len(arr)):
            try:
                result = next(iterator)
                results.append(result)
            except StopIteration:
                break  
            except TimeoutError as error:
                print("function took longer than seconds", error.args[1])
    return results

## Convert mxl to midi

In [11]:
df = pd.read_csv(source_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,7,8,10,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,song_url,ht_bpm,ht_time_signature,midi,genres,artist,md5,section,ht_key,source,parts,midi_title,title,ht_offset,ht_mode,mxl
0,https://www.hooktheory.com/theorytab/view/wayn...,128.0,4.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,NaN,wayne-sharpe,bf1f29e5ff84e3e93e37fb873bfb590e,chorus,C,hooktheory,"intro,chorus",yu-gi-oh3,yu-gi-oh-theme-song,0.0,1.0,NaN
1,https://www.hooktheory.com/theorytab/view/wayn...,85.0,3.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...,NaN,wayne-sharpe,055f80ad67f64edb14a85ca8fbfe8c29,intro,C,hooktheory,"intro,chorus",yu-gi-oh,yu-gi-oh-theme-song,0.0,1.0,NaN
2,https://www.hooktheory.com/theorytab/view/what...,96.0,4.0,midi_sources/hooktheory/pianoroll/w/what-a-day...,Jazz,what-a-day,197f96f5d181f6ce1e2c5ab04ac1ff87,chorus,D,hooktheory,chorus,kiefer,kiefer,-5.0,6.0,NaN
3,https://www.hooktheory.com/theorytab/view/whit...,152.0,4.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,"J-Pop,Pop",whiteflame,9e7ce13a35f1314423a9a6d5a5287a4a,pre-chorus,D,hooktheory,"verse,pre-chorus,chorus",senbonzakura - pre-Pre-Chorus,senbonzakura,-5.0,6.0,NaN
4,https://www.hooktheory.com/theorytab/view/whit...,152.0,4.0,midi_sources/hooktheory/pianoroll/w/whiteflame...,"J-Pop,Pop",whiteflame,d5aaf79d0989222f1362f9f46c540a27,verse,D,hooktheory,"verse,pre-chorus,chorus",Senbonzakura,senbonzakura,-5.0,6.0,NaN


In [12]:
all_records = df.to_dict(orient='records')

In [13]:
def mxl2midi_func(metadata):
    result = metadata.copy()
    if not isinstance(result.get('mxl'), str): return result

    input_path = version_path/metadata['mxl']
    out_file = Path(str(metadata['mxl']).replace('midi_sources/', 'midi_sources/from_mxl/')).with_suffix('.mid')
    output_path = version_path/out_file
    
    if not output_path.exists():
        try:
            output_path.parent.mkdir(parents=True, exist_ok=True)
            stream = file2stream(input_path)
            stream.write('midi', fp=output_path)
        except Exception:
            print(traceback.format_exc())
            return None
        
    result['midi'] = out_file
    return result

In [14]:
# # sanity check
# import random
# for r in random.sample(mxl_records, 500):
#     mxl2midi_func(r)
#     print(r)

In [15]:
# processed = process_all(mxl2midi_func, all_records, timeout=10)

In [14]:
processed = process_all(mxl2midi_func, all_records, timeout=300)

In [15]:
from src.data_sources import arr2csv
arr2csv(processed, out_csv)
df = pd.read_csv(out_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,3,5,6,8,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,genres,artist,ht_key,midi_title,ht_mode,mxl,song_url,ht_offset,parts,title,ht_bpm,ht_time_signature,section,midi,source,md5
0,NaN,wayne-sharpe,C,yu-gi-oh3,1.0,NaN,https://www.hooktheory.com/theorytab/view/wayn...,0.0,"intro,chorus",yu-gi-oh-theme-song,128.0,4.0,chorus,midi_sources/hooktheory/pianoroll/w/wayne-shar...,hooktheory,bf1f29e5ff84e3e93e37fb873bfb590e
1,NaN,wayne-sharpe,C,yu-gi-oh,1.0,NaN,https://www.hooktheory.com/theorytab/view/wayn...,0.0,"intro,chorus",yu-gi-oh-theme-song,85.0,3.0,intro,midi_sources/hooktheory/pianoroll/w/wayne-shar...,hooktheory,055f80ad67f64edb14a85ca8fbfe8c29
2,Jazz,what-a-day,D,kiefer,6.0,NaN,https://www.hooktheory.com/theorytab/view/what...,-5.0,chorus,kiefer,96.0,4.0,chorus,midi_sources/hooktheory/pianoroll/w/what-a-day...,hooktheory,197f96f5d181f6ce1e2c5ab04ac1ff87
3,"J-Pop,Pop",whiteflame,D,senbonzakura - pre-Pre-Chorus,6.0,NaN,https://www.hooktheory.com/theorytab/view/whit...,-5.0,"verse,pre-chorus,chorus",senbonzakura,152.0,4.0,pre-chorus,midi_sources/hooktheory/pianoroll/w/whiteflame...,hooktheory,9e7ce13a35f1314423a9a6d5a5287a4a
4,"J-Pop,Pop",whiteflame,D,Senbonzakura,6.0,NaN,https://www.hooktheory.com/theorytab/view/whit...,-5.0,"verse,pre-chorus,chorus",senbonzakura,152.0,4.0,verse,midi_sources/hooktheory/pianoroll/w/whiteflame...,hooktheory,d5aaf79d0989222f1362f9f46c540a27


### Encoding midi to numpy

In [14]:
df = pd.read_csv(out_csv); df.head()

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2,3,5,6,8,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,genres,artist,ht_key,midi_title,ht_mode,mxl,song_url,ht_offset,parts,title,ht_bpm,ht_time_signature,section,midi,source,md5
0,NaN,wayne-sharpe,C,yu-gi-oh3,1.0,NaN,https://www.hooktheory.com/theorytab/view/wayn...,0.0,"intro,chorus",yu-gi-oh-theme-song,128.0,4.0,chorus,midi_sources/hooktheory/pianoroll/w/wayne-shar...,hooktheory,bf1f29e5ff84e3e93e37fb873bfb590e
1,NaN,wayne-sharpe,C,yu-gi-oh,1.0,NaN,https://www.hooktheory.com/theorytab/view/wayn...,0.0,"intro,chorus",yu-gi-oh-theme-song,85.0,3.0,intro,midi_sources/hooktheory/pianoroll/w/wayne-shar...,hooktheory,055f80ad67f64edb14a85ca8fbfe8c29
2,Jazz,what-a-day,D,kiefer,6.0,NaN,https://www.hooktheory.com/theorytab/view/what...,-5.0,chorus,kiefer,96.0,4.0,chorus,midi_sources/hooktheory/pianoroll/w/what-a-day...,hooktheory,197f96f5d181f6ce1e2c5ab04ac1ff87
3,"J-Pop,Pop",whiteflame,D,senbonzakura - pre-Pre-Chorus,6.0,NaN,https://www.hooktheory.com/theorytab/view/whit...,-5.0,"verse,pre-chorus,chorus",senbonzakura,152.0,4.0,pre-chorus,midi_sources/hooktheory/pianoroll/w/whiteflame...,hooktheory,9e7ce13a35f1314423a9a6d5a5287a4a
4,"J-Pop,Pop",whiteflame,D,Senbonzakura,6.0,NaN,https://www.hooktheory.com/theorytab/view/whit...,-5.0,"verse,pre-chorus,chorus",senbonzakura,152.0,4.0,verse,midi_sources/hooktheory/pianoroll/w/whiteflame...,hooktheory,d5aaf79d0989222f1362f9f46c540a27


In [15]:
all_records = df.to_dict(orient='records'); len(all_records)

197426

In [16]:
# mxl_records = df.loc[df.source.isin(['classical_archives', 'hooktheory_c'])].to_dict(orient='records')
# mxl_records = df.loc[df.source.isin(['ecomp', 'wikifonia', 'classical_piano', 'musescore'])].to_dict(orient='records')

In [17]:
def transform_func(metadata):
    result = metadata.copy()
    start = time.time()
    
    # Part 1. Compress tracks/instruments
    if not isinstance(metadata.get('midi'), str): return None
    
    input_path = version_path/metadata['midi']
    extension = input_path.suffix.lower()
    if not input_path.exists(): 
        print('Input path does not exist:', input_path, metadata)
        return None
    
    # Get outfile and check if it exists
    out_file = Path(str(input_path).replace(f'/{source_dir}/', f'/{out_dir}/'))
    out_file = out_file.with_suffix('.npy')
    out_file.parent.mkdir(parents=True, exist_ok=True)
    if out_file.exists(): 
        result['numpy'] = str(out_file.relative_to(version_path))
        return result
    
#     # DEBUG
#     return None
    
    try: input_file = compress_midi_file(input_path, cutoff=cutoff) # remove non note tracks and standardize instruments
    except music21.midi.MidiException as e:
        print('Error parsing midi', input_path, e)
        return None
    if not input_file: return None
        
    # Part 2. Compress rests and long notes
    stream = file2stream(input_file) # 1.
    try:
        chordarr = stream2chordarr(stream, max_dur=max_dur, flat=False) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
#     chordarr = stream2chordarr(stream, max_dur=max_dur, flat=(extension != '.mid')) # 2. max_dur = quarter_len * sample_freq (4). 128 = 8 bars
    except Exception as e:
        print('Could not encode to chordarr:', input_path, e)
#         print(traceback.format_exc())
        return None
    
    chord_trim = trim_chordarr_rests(chordarr)
    chord_short = shorten_chordarr_rests(chord_trim)
    delta_trim = chord_trim.shape[0] - chord_short.shape[0]
    if delta_trim > 300: 
        print(f'Removed {delta_trim} rests from {input_path}. Skipping song')
        return None
    chordarr = chord_short
    
    # Part 3. Chord array to numpy
    seq = chordarr2seq(chordarr)
    if len(seq) < 32:
        print('Sequence too short:', len(seq), input_path)
        return None
    
#     category = ENC_OFFSET + max_dur + sources.index(metadata['source']) + 1
    category = source2encidx(metadata['source'], max_dur=max_dur)
    npenc = seq2npenc(seq, num_comps=num_comps, category=category)
    if (npenc[1:,1] > max_dur+ENC_OFFSET).any(): 
        print('npenc exceeds max 128 duration:', input_path)
        return None
    
    if ((npenc[...,0] > ENC_OFFSET) & ((npenc[...,0] < 12+ENC_OFFSET) | (npenc[...,0] >= 127-12+ENC_OFFSET))).any(): 
        print('npenc out of note range 12 - 116:', input_path)
        return None
    
    
    np.save(out_file, npenc)
    
    elapsed_time = time.time() - start
    result['sec_elapsed_tfm'] = elapsed_time
    if elapsed_time > 60:
        print('Super long file:', elapsed_time, input_path)
    
    result['numpy'] = str(out_file.relative_to(version_path))
    return result

In [18]:
def try_transform_func(metadata):
    try:
        return transform_func(metadata)
    except Exception:
        print(traceback.format_exc())
        return None

In [19]:
# # # sanity check
# import random
# for r in random.sample(mxl_records, 500):
#     transform_func(r)
        

In [20]:
### AS TOD: Fix 'info channel is not channel 0' error

In [21]:
# processed = process_all(try_transform_func, mxl_records)

In [22]:
processed = process_all(try_transform_func, all_records, timeout=300)

Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transform_func
    return transform_func(metadata)
  File "<ipython-input-17-8ce624b49e4b>", line 32, in transform_func
    stream = file2stream(input_file) # 1.
  File "../../src/midi_data.py", line 78, in file2stream
    if isinstance(fp, music21.midi.MidiFile): return music21.midi.translate.midiFileToStream(fp)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 2115, in midiFileToStream
    **keywords)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 1905, in midiTracksToStreams
    p = s.getElementsByClass('Stream')[0]
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/stream/iterator.py", line 328, in __getitem__
    e = fe[k]
IndexError: list index out of range



Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transform_func
    return transform_func(metadata)
  File "<ipython-input-17-8ce624b49e4b>", line 32, in transform_func
    stream = file2stream(input_file) # 1.
  File "../../src/midi_data.py", line 78, in file2stream
    if isinstance(fp, music21.midi.MidiFile): return music21.midi.translate.midiFileToStream(fp)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 2115, in midiFileToStream
    **keywords)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 1905, in midiTracksToStreams
    p = s.getElementsByClass('Stream')[0]
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/stream/iterator.py", line 328, in __getitem__
    e = fe[k]
IndexError: list index out of range

Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transform


Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transform_func
    return transform_func(metadata)
  File "<ipython-input-17-8ce624b49e4b>", line 32, in transform_func
    stream = file2stream(input_file) # 1.
  File "../../src/midi_data.py", line 78, in file2stream
    if isinstance(fp, music21.midi.MidiFile): return music21.midi.translate.midiFileToStream(fp)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 2115, in midiFileToStream
    **keywords)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 1905, in midiTracksToStreams
    p = s.getElementsByClass('Stream')[0]
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/stream/iterator.py", line 328, in __getitem__
    e = fe[k]
IndexError: list index out of range

Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transfor


Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transform_func
    return transform_func(metadata)
  File "<ipython-input-17-8ce624b49e4b>", line 32, in transform_func
    stream = file2stream(input_file) # 1.
  File "../../src/midi_data.py", line 78, in file2stream
    if isinstance(fp, music21.midi.MidiFile): return music21.midi.translate.midiFileToStream(fp)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 2115, in midiFileToStream
    **keywords)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 1905, in midiTracksToStreams
    p = s.getElementsByClass('Stream')[0]
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/stream/iterator.py", line 328, in __getitem__
    e = fe[k]
IndexError: list index out of range

Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transfor

Sequence too short: 25 data/midi/v13/midi_sources/hooktheory/pianoroll/k/koji-kondo/link-to-the-past---minigame-theme/intro_key_original.mid
Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transform_func
    return transform_func(metadata)
  File "<ipython-input-17-8ce624b49e4b>", line 32, in transform_func
    stream = file2stream(input_file) # 1.
  File "../../src/midi_data.py", line 78, in file2stream
    if isinstance(fp, music21.midi.MidiFile): return music21.midi.translate.midiFileToStream(fp)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 2115, in midiFileToStream
    **keywords)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 1905, in midiTracksToStreams
    p = s.getElementsByClass('Stream')[0]
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/stream/iterator.py", line 328, in __getitem__
    e = fe[

...


Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transform_func
    return transform_func(metadata)
  File "<ipython-input-17-8ce624b49e4b>", line 32, in transform_func
    stream = file2stream(input_file) # 1.
  File "../../src/midi_data.py", line 78, in file2stream
    if isinstance(fp, music21.midi.MidiFile): return music21.midi.translate.midiFileToStream(fp)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 2115, in midiFileToStream
    **keywords)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 1905, in midiTracksToStreams
    p = s.getElementsByClass('Stream')[0]
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/stream/iterator.py", line 328, in __getitem__
    e = fe[k]
IndexError: list index out of range

Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transfor

Sequence too short: 17 data/midi/v13/midi_sources/hooktheory/pianoroll/t/the-beatles/a-hard-days-night/intro-and-verse_key_cmajor.mid
Sequence too short: 17 data/midi/v13/midi_sources/hooktheory/pianoroll/t/they-might-be-giants/everything-right-is-wrong-again/chorus-lead-out_key_cmajor.mid
function took longer than seconds 30
Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transform_func
    return transform_func(metadata)
  File "<ipython-input-17-8ce624b49e4b>", line 32, in transform_func
    stream = file2stream(input_file) # 1.
  File "../../src/midi_data.py", line 78, in file2stream
    if isinstance(fp, music21.midi.MidiFile): return music21.midi.translate.midiFileToStream(fp)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 2115, in midiFileToStream
    **keywords)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 1905, in midiTrack


Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transform_func
    return transform_func(metadata)
  File "<ipython-input-17-8ce624b49e4b>", line 32, in transform_func
    stream = file2stream(input_file) # 1.
  File "../../src/midi_data.py", line 78, in file2stream
    if isinstance(fp, music21.midi.MidiFile): return music21.midi.translate.midiFileToStream(fp)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 2115, in midiFileToStream
    **keywords)
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/midi/translate.py", line 1905, in midiTracksToStreams
    p = s.getElementsByClass('Stream')[0]
  File "/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/music21/stream/iterator.py", line 328, in __getitem__
    e = fe[k]
IndexError: list index out of range

Sequence too short: 17 data/midi/v13/midi_sources/hooktheory/pianoroll/m/men-at-work/land-down-under


function took longer than seconds 30
Could not encode to chordarr: data/midi/v13/midi_sources/130k_reddit/W/W/warmgun.mid index 815 is out of bounds for axis 0 with size 815
Could not encode to chordarr: data/midi/v13/midi_sources/130k_reddit/R/R/roxanne03.mid index 127 is out of bounds for axis 2 with size 127
Could not encode to chordarr: data/midi/v13/midi_sources/130k_reddit/R/R/redroost.mid index 1147 is out of bounds for axis 0 with size 1147
Could not encode to chordarr: data/midi/v13/midi_sources/130k_reddit/T/T/tropicalrain.mid index 1399 is out of bounds for axis 0 with size 1399
Traceback (most recent call last):
  File "<ipython-input-18-c95459e3fde9>", line 3, in try_transform_func
    return transform_func(metadata)
  File "<ipython-input-17-8ce624b49e4b>", line 32, in transform_func
    stream = file2stream(input_file) # 1.
  File "../../src/midi_data.py", line 78, in file2stream
    if isinstance(fp, music21.midi.MidiFile): return music21.midi.translate.midiFileToStrea

function took longer than seconds 30
function took longer than seconds 30
function took longer than seconds 30
function took longer than seconds 30
npenc out of note range 12 - 116: data/midi/v13/midi_sources/130k_reddit/T/T/thuong_anh_mat_doi_mat_cho.mid
function took longer than seconds 30
function took longer than seconds 30
Could not encode to chordarr: data/midi/v13/midi_sources/130k_reddit/R/R/Rosanna4.mid index 1619 is out of bounds for axis 0 with size 1619
function took longer than seconds 30
function took longer than seconds 30
npenc out of note range 12 - 116: data/midi/v13/midi_sources/130k_reddit/R/R/rammstein-engel.mid
npenc out of note range 12 - 116: data/midi/v13/midi_sources/130k_reddit/R/R/RONDO.MID
function took longer than seconds 30
Could not encode to chordarr: data/midi/v13/midi_sources/130k_reddit/R/R/ragtime3.mid index 2911 is out of bounds for axis 0 with size 2911
npenc out of note range 12 - 116: data/midi/v13/midi_sources/130k_reddit/T/T/THE SERVANT.Liquef

In [24]:
from src.data_sources import arr2csv

In [27]:
processed_tmp = processed

In [28]:
processed = [p for p in processed_tmp if p is not None]

In [29]:
arr2csv(processed, out_csv); len(processed)

189846

In [30]:
df = pd.read_csv(out_csv)

/home/ubuntu/anaconda3/envs/midi/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (0,1,6,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
df.head()

,midi_title,parts,title,ht_bpm,md5,artist,section,ht_offset,numpy,ht_mode,mxl,source,genres,ht_key,song_url,ht_time_signature,midi
0,yu-gi-oh3,"intro,chorus",yu-gi-oh-theme-song,128.0,bf1f29e5ff84e3e93e37fb873bfb590e,wayne-sharpe,chorus,0.0,midi_encode/hooktheory/pianoroll/w/wayne-sharp...,1.0,NaN,hooktheory,NaN,C,https://www.hooktheory.com/theorytab/view/wayn...,4.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...
1,yu-gi-oh,"intro,chorus",yu-gi-oh-theme-song,85.0,055f80ad67f64edb14a85ca8fbfe8c29,wayne-sharpe,intro,0.0,midi_encode/hooktheory/pianoroll/w/wayne-sharp...,1.0,NaN,hooktheory,NaN,C,https://www.hooktheory.com/theorytab/view/wayn...,3.0,midi_sources/hooktheory/pianoroll/w/wayne-shar...
2,kiefer,chorus,kiefer,96.0,197f96f5d181f6ce1e2c5ab04ac1ff87,what-a-day,chorus,-5.0,midi_encode/hooktheory/pianoroll/w/what-a-day/...,6.0,NaN,hooktheory,Jazz,D,https://www.hooktheory.com/theorytab/view/what...,4.0,midi_sources/hooktheory/pianoroll/w/what-a-day...
3,senbonzakura - pre-Pre-Chorus,"verse,pre-chorus,chorus",senbonzakura,152.0,9e7ce13a35f1314423a9a6d5a5287a4a,whiteflame,pre-chorus,-5.0,midi_encode/hooktheory/pianoroll/w/whiteflame/...,6.0,NaN,hooktheory,"J-Pop,Pop",D,https://www.hooktheory.com/theorytab/view/whit...,4.0,midi_sources/hooktheory/pianoroll/w/whiteflame...
4,Senbonzakura,"verse,pre-chorus,chorus",senbonzakura,152.0,d5aaf79d0989222f1362f9f46c540a27,whiteflame,verse,-5.0,midi_encode/hooktheory/pianoroll/w/whiteflame/...,6.0,NaN,hooktheory,"J-Pop,Pop",D,https://www.hooktheory.com/theorytab/view/whit...,4.0,midi_sources/hooktheory/pianoroll/w/whiteflame...
